In [ ]:
import requests
import pandas as pd
from IPython.display import clear_output, display
import datetime

In [ ]:

URL_2023 = "https://eventresults-api.sporthive.com/api/events/7117550207329210624/races/487268/classifications/search?count=50&offset={}"
URL_2022 = "https://eventresults-api.sporthive.com/api/events/6981282006063051264/races/483567/classifications/search?count=50&offset={}"


URL = URL_2022
YEAR=2022
NAME="amsterdam"

In [ ]:
keys=['bib','name', 'countryCode',  'gender', 'category','chipTime']

pg_index=1

count=0

df_final=None

while True:
    res = requests.get(URL.format(count), timeout=30)
    data=res.json()
    
    if 'fullClassifications' not in data or len(data['fullClassifications'])==0:
        break

    records = [ { key: rec['classification'].get(key,'') for key in keys } for rec in data['fullClassifications']]
    df=pd.DataFrame(records)
    count+=len(records)
    
    clear_output()
    display(f"{pg_index} - {count}")


    if df_final is None:
        df_final=df
    else:
        df_final=pd.concat([df_final, df])

    pg_index+=1


df_final.shape

In [ ]:
df_final.reset_index(inplace=True)
df_final['age']= df_final.category.str[1:]
df_final['sex']=df_final.gender.map({1:'M', 2:'F'})

In [ ]:
df_final.head()

In [ ]:
df_final.to_csv(f"{NAME}_{YEAR}_full.csv")
df_final[['sex','age','chipTime']].to_csv(f"{NAME}_{YEAR}.csv")

In [ ]:
df_final[['age','sex']].groupby('sex', dropna=False).count()

In [ ]:
df_final.gender.value_counts()